# Classification de terrains

## Préparation des données

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from argparse import ArgumentParser

import os
import matplotlib.pyplot as plt

On initialise les variables de base

In [1]:
BATCH_SIZE = 128
EPOCHS = 15
IMG_HEIGHT = 600
IMG_WIDTH = 600

Afin de pouvoir modifier ces paramètres, on ajoute à notre script deux arguments facultatifs permettant la modification du nombre d'epochs et la taille du batch.

In [ ]:
parser = ArgumentParser()
parser.add_argument("-e", "--epochs", dest="EPOCHS", help="Specify the number of epochs", type=int)
parser.add_argument("-b", "--batch", dest="BATCH_SIZE", help="Specify the batch size", type=int)
args = parser.parse_args()
if args.EPOCHS:
    EPOCHS = args.EPOCHS
if args.BATCH_SIZE:
    BATCH_SIZE = args.BATCH_SIZE

print("EPOCHS = %d" % EPOCHS)
print("BATCH_SIZE = %d" % BATCH_SIZE)

On crée ensuite notre ImageDataGenerator qui s'occuppe de lire des images sur disque et les prétraiter dans les tenseurs appropriés. Il mettra également en place des générateurs qui convertiront ces images en lots de tenseurs, ce qui sera utile lors de la formation du réseau.

In [ ]:
train_image_generator = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=45,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True,
    zoom_range=0.5
)

validation_image_generator = ImageDataGenerator(
    rescale=1. / 255
)

Après avoir défini les générateurs d'images de formation et de validation, la méthode flow_from_directory charge les images du disque, applique un redimensionnement et redimensionne les images dans les dimensions requises.

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(
    directory='data/training/',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

val_data_gen = validation_image_generator.flow_from_directory(
    directory='data/validation/',
    target_size=(600, 600),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

## Création du modèle

Une fois les données traîtées, on peut construire notre modèle
Celui-ci se compose de trois blocs de convolution avec une couche de pool maximum dans chacun d'eux. Il y a une couche entièrement connectée avec 512 unités sur le dessus qui est activée par une fonction d'activation de la réluctance. Le modèle produit des probabilités de classe basées sur une classification binaire par la fonction d'activation sigmoïde.

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

Le modèle est ensuite compilé, nous avons choisi l'optimiseur ADAM et la fonction de perte "binary_crossentropy". 
Pour visualiser la précision de l'entraînement et de la validation pour chaque epoch d'entraînement, on passe l'argument "metrics".

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Il est possible à tout moment de voir les différentes couches que contient notre modèle

In [ ]:
model.summary()

## Entraînement du modèle

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=BATCH_SIZE
)

Une fois notre générateur entraîné, on récupère les résultats

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

On utilise ensuite les résultats de notre entraînement afin de réaliser un graphique récapitulatif de la précision de notre classificateur

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

Ainsi que les pertes

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

On sauvegarde ensuite notre graphique ainsi que notre modèle entraîné pour des utilisations futures et on finit par afficher le graphique.

In [ ]:
filename = "../models/MODEL_E" + str(EPOCHS) + "_B" + str(BATCH_SIZE)

plt.savefig(filename + ".png")
model.save(filename + ".h5")
plt.show()

## Classification de nos données
On crée également un jeu de test et on fait prédire à notre modèle entraîné la classe à laquelle il pense que l'image appartient ("terrain" ou "others").

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from argparse import ArgumentParser

import os
import numpy as np


# FUNCTIONS
def test_images(model, path):
    for file in os.listdir(path):
        if file.endswith(('.jpg', '.png', '.tif')):
            img = image.load_img(os.path.join(path, file))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            result = model.predict_classes(img)
            print("[" + ("Other", "Terrain")[result[0][0] == 1] + "] " + file + " (" + str(result[0][0]) + ")")


# GET ARGS
parser = ArgumentParser()
parser.add_argument("model", help="Model file to load", metavar="FILE")
args = parser.parse_args()
if not os.path.exists(args.model):
    print("Erreur: veuillez spécifier un nom de modèle à charger valide!")
    exit(1)

# On charge notre modèle existant
model = load_model(args.model)

# On le recompile
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# PART 3: TEST MODEL
print("--------------------- TERRAINS ---------------------")
test_images(model, 'data/test/terrains/')

print("--------------------- NOT A TERRAIN ---------------------")
test_images(model, 'data/test/others/')